# CNN for Cifar-10

In [26]:
import tensorflow as tf
import numpy as np
import pickle

def unpickle(file):
    with open(file, 'rb') as fo:
        d = pickle.load(fo, encoding='latin1')
    return d
    
def onehot(labels):
    ''' one-hot encoding'''
    n_sample = len(labels)
    n_class = max(labels) + 1
    onehot_labels = np.zeros((n_sample, n_class))
    onehot_labels[np.arange(n_sample), labels] = 1

    return onehot_labels

def reformat(dataset):
    image_size=np.sqrt(dataset.shape[1]/3)
    dataset = dataset.reshape((dataset.shape[0], image_size.astype(int), image_size.astype(int), num_channels)).astype(np.float32)
    #labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  
    return dataset

data1 = unpickle('cifar-10-batches-py/data_batch_1')
data2 = unpickle('cifar-10-batches-py/data_batch_2')
data3 = unpickle('cifar-10-batches-py/data_batch_3')
data4 = unpickle('cifar-10-batches-py/data_batch_4')
data5 = unpickle('cifar-10-batches-py/data_batch_5')

train_dataset = np.concatenate((data1['data'], data2['data'], data3['data'], data4['data'], data5['data']), axis=0)
label = np.concatenate((data1['labels'], data2['labels'], data3['labels'], data4['labels'], data5['labels']), axis=0)
train_labels = onehot(label)
train_dataset = train_dataset / 255.0
print ("befor reformat train shape: ", train_dataset.shape)
train_dataset = reformat(train_dataset)
print ("after reformat train shape: ", train_dataset.shape)

test = unpickle('cifar-10-batches-py/test_batch')
test_dataset = test['data'] / 255.0
test_labels = onehot(test['labels'])

test_dataset= reformat(test_dataset)

n_sample = train_dataset.shape[0]
num_channels = 3
num_labels = 10
batch_size = 16
patch_size = 3
depth = 16
num_hidden = 64
beta = 0.01
image_size = 32
total_epoch = 10
learning_rate = 0.001

x = tf.placeholder(tf.float32, [None, image_size, image_size, num_channels])
y = tf.placeholder(tf.float32, [None, num_labels])

# Parameters
layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
layer1_biases = tf.Variable(tf.zeros([depth]))
layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
layer3_weights = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))

# Define Model
def model(input_image):
    conv1 = tf.nn.conv2d(input_image, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden1 = tf.nn.relu(conv1 + layer1_biases)
    conv2 = tf.nn.conv2d(hidden1, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden2 = tf.nn.relu(conv2 + layer2_biases)
    shape = hidden2.get_shape().as_list()
    reshape = tf.reshape(hidden2, [-1, shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases

# Training computation.
logits = model(x)
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=logits)\
                      +beta*tf.nn.l2_loss(layer1_weights)#regularization
                      +beta*tf.nn.l2_loss(layer2_weights)
                      +beta*tf.nn.l2_loss(layer3_weights)
                      +beta*tf.nn.l2_loss(layer4_weights)
                     
                     )

# Optimizer
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)

prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print('Initialized')
    for epoch in range(total_epoch):
        # calculate total batch within a epoch
        num_steps =  train_dataset.shape[0] // batch_size
        avg_cost = 0 
        avg_acc = 0.0
        for step in range(num_steps):
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
            batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
            batch_labels = train_labels[offset:(offset + batch_size), :]
            feed_dict = { x: batch_data, y: batch_labels}
            _, l, train_accuracy_ = sess.run([optimizer, loss, accuracy], feed_dict=feed_dict)
            avg_cost = avg_cost + (l/num_steps)
            avg_acc = avg_acc + (train_accuracy_/num_steps)
        print('loss at epoch {}:\ncost: {}, accuracy: {}'.format(epoch, avg_cost, avg_acc))
            
    '''
    Testing Part
    ''' 
    
    print('Testing......')
    feed_dict = { x: test_dataset, y: test_labels}
    test_accuracy_ = sess.run(accuracy, feed_dict=feed_dict)
    print('Test accuracy: {}'.format(test_accuracy_))



befor reformat train shape:  (50000, 3072)
after reformat train shape:  (50000, 32, 32, 3)
Initialized
loss at epoch 0:
cost: 2.2278483298110894, accuracy: 0.3409999999999969
loss at epoch 1:
cost: 1.773177111053468, accuracy: 0.4282999999999944
loss at epoch 2:
cost: 1.7331751992797846, accuracy: 0.44525999999999505
loss at epoch 3:
cost: 1.7127022282791162, accuracy: 0.4557999999999941
loss at epoch 4:
cost: 1.6956869562911976, accuracy: 0.4662799999999943
loss at epoch 5:
cost: 1.681262268142698, accuracy: 0.4733399999999935
loss at epoch 6:
cost: 1.6714608768272368, accuracy: 0.4776399999999941
loss at epoch 7:
cost: 1.6660958282279934, accuracy: 0.48061999999999294
loss at epoch 8:
cost: 1.6620354361915612, accuracy: 0.48183999999999305
loss at epoch 9:
cost: 1.6580306155013993, accuracy: 0.4844799999999936
Testing......
Test accuracy: 0.47839999198913574


# Cat and Dog image classification

In [32]:
#每一張照片大小不一樣
import numpy as np
from os import listdir
from random import shuffle
from PIL import Image                   #python讀取照片套件

image_path = ["./dataset/cat_dog/cat", "./dataset/cat_dog/dog"]

def preprocess_to_data_file(data_dir_list, ratio=0.8):

    total_list = [] 
   
    '''
        create all images into (image path, label) format and write to total_data.txt 
    ''' 
    with open('total_data.txt', 'w') as f:                                                #cat:0, dog:1
        for index, data_dir in enumerate(data_dir_list):
            for filename in listdir(data_dir):
                #print("{} {}".format(data_dir_list[index]+'/'+filename, index))
                f.write('{} {}\n'.format(data_dir_list[index]+'/'+filename.replace(' ',''), index)) #不懂為何要.replace(' ','')
                total_list.append(data_dir_list[index]+'/'+filename.replace(' ','')+' '+str(index))

    #print(total_list)
    '''
        shuffle total_list
    ''' 
    shuffle(total_list)

    '''
        split total_list to train_list and test_list. 
        write train_list/test_list into train_data.txt/test_data.txt
    ''' 
    train_list = total_list[:int(ratio*len(total_list))]
    test_list = total_list[int(ratio*len(total_list)):]

    with open('train_data.txt', 'w') as f:
        for i in train_list:
            f.write(i+'\n')

    with open('test_data.txt', 'w') as f:
        for i in test_list:
            f.write(i+'\n')

preprocess_to_data_file(image_path, 0.8)

In [88]:
import numpy as np
from random import shuffle
from PIL import Image
import tensorflow as tf
import os
    
image_size = 100
num_labels = 2
num_channels = 3 

def load_txt(file_path):                      #擷取image path及label
    file=open(file_path)
    lines=file.readlines()
    paths=[]
    labels=[]
    for line in lines:
        line=line.split()
        paths.append(line[0])
        labels.append(int(line[1]))
    return paths, labels

def one_hot(values):
    values = np.asarray(values)
    n_values = np.max(values) + 1
    return np.eye(n_values)[values]

def load_batch_data(paths, labels):
    images = []
    for path in paths:
        image = Image.open(path)
        
        #resize and normalize
        image = image.resize((image_size, image_size))
        image = np.asarray(image)/255.0
        images.append(image)
           
    #one-hot
    one_hot_labels = one_hot(labels)
    return images, one_hot_labels

'''
def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size, image_size, num_channels)).astype(np.float32)
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels
'''

train_data, train_labels = load_txt("train_data.txt")
test_data, test_labels = load_txt("train_data.txt")

#imgs, labels = load_batch_data(train_data[0:5], train_labels[0:5])
#print (imgs[0].shape)
#print (labels[0].shape)
#print (train_data)
#print (train_labels)


In [94]:
batch_size = 64
patch_size = 3
depth = 32
num_hidden = 64
beta = 0.01
x = tf.placeholder(tf.float32, [None, image_size, image_size, num_channels])
y = tf.placeholder(tf.float32, [None, num_labels])

# Parameters
layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
layer1_biases = tf.Variable(tf.zeros([depth]))
layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
layer3_weights = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))

# Define Model
def model(input_image):
    conv1 = tf.nn.conv2d(input_image, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden1 = tf.nn.relu(conv1 + layer1_biases)
    conv2 = tf.nn.conv2d(hidden1, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden2 = tf.nn.relu(conv2 + layer2_biases)
    shape = hidden2.get_shape().as_list()
    reshape = tf.reshape(hidden2, [-1, shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases

# Training computation.
logits = model(x)
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=logits)\
                      +beta*tf.nn.l2_loss(layer1_weights)                                   #regularization
                      +beta*tf.nn.l2_loss(layer2_weights)
                      +beta*tf.nn.l2_loss(layer3_weights)
                      +beta*tf.nn.l2_loss(layer4_weights)
                     
                     )

# Optimizer
optimizer = tf.train.AdamOptimizer(1e-4).minimize(loss)

prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))

saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print('Initialized')
    for epoch in range(total_epoch):
        # calculate total batch within a epoch
        num_steps =  len(train_data) // batch_size
        avg_cost = 0 
        avg_acc = 0.0
        for step in range(num_steps):          
            offset = (step * batch_size) % (len(train_labels) - batch_size)
            batch_train_data_paths = train_data[offset:(offset + batch_size)]
            batch_train_labels_paths = train_labels[offset:(offset + batch_size)]
            batrch_train_data, batch_train_labels = load_batch_data(batch_train_data_paths, batch_train_labels_paths)
            
            feed_dict = { x: batrch_train_data, y: batch_train_labels}
            _, l, train_accuracy_ = sess.run([optimizer, loss, accuracy], feed_dict=feed_dict)
            
            avg_cost = avg_cost + (l/num_steps)
            avg_acc = avg_acc + (train_accuracy_/num_steps)
            
            if (step % 100 == 0):
                saver.save(sess, "train_model/model.ckpt")
                print('step={}, loss={}, accuracy={}'.format(step, l, train_accuracy_))
                test_batch_data, test_batch_labels = load_batch_data(test_data[:2000], test_labels[:2000])
                feed_dict = { x: test_batch_data, y: test_batch_labels}
                test_accuracy_ = sess.run(accuracy, feed_dict=feed_dict)
                print('test accuracy = {}'.format(test_accuracy_))

            
        print('loss at epoch {}:\ncost: {}, accuracy: {}'.format(epoch, avg_cost, avg_acc))
            


Initialized
step=0, loss=32.20766067504883, accuracy=0.4375
test accuracy = 0.5180000066757202
step=100, loss=24.193283081054688, accuracy=0.375
test accuracy = 0.5040000081062317
step=200, loss=22.487510681152344, accuracy=0.3125
test accuracy = 0.5070000290870667
step=300, loss=20.93109703063965, accuracy=0.125
test accuracy = 0.5210000276565552
step=400, loss=19.77070426940918, accuracy=0.4375
test accuracy = 0.5120000243186951
step=500, loss=18.45924186706543, accuracy=0.4375
test accuracy = 0.5214999914169312
step=600, loss=17.191646575927734, accuracy=0.6875
test accuracy = 0.5115000009536743
step=700, loss=16.313167572021484, accuracy=0.6875
test accuracy = 0.5404999852180481
step=800, loss=15.501067161560059, accuracy=0.625
test accuracy = 0.5189999938011169
step=900, loss=14.899615287780762, accuracy=0.4375
test accuracy = 0.5490000247955322
step=1000, loss=14.150572776794434, accuracy=0.3125
test accuracy = 0.5260000228881836
step=1100, loss=14.119354248046875, accuracy=0.25


test accuracy = 0.6050000190734863
step=100, loss=2.945345878601074, accuracy=0.75
test accuracy = 0.6754999756813049
step=200, loss=2.940401554107666, accuracy=0.625
test accuracy = 0.6545000076293945
step=300, loss=2.7718799114227295, accuracy=0.75
test accuracy = 0.6955000162124634
step=400, loss=2.983983039855957, accuracy=0.5
test accuracy = 0.6535000205039978
step=500, loss=2.893315315246582, accuracy=0.6875
test accuracy = 0.7070000171661377
step=600, loss=2.5638461112976074, accuracy=0.875
test accuracy = 0.6974999904632568
step=700, loss=2.7237792015075684, accuracy=0.625
test accuracy = 0.6825000047683716
step=800, loss=2.689957618713379, accuracy=0.5625
test accuracy = 0.6654999852180481
step=900, loss=2.563957691192627, accuracy=0.625
test accuracy = 0.7135000228881836
step=1000, loss=2.4590606689453125, accuracy=0.625
test accuracy = 0.7110000252723694
step=1100, loss=2.807037115097046, accuracy=0.375
test accuracy = 0.6549999713897705
step=1200, loss=2.326925277709961, ac

# use pb file to store model

In [5]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.python.framework import graph_util

mnist = input_data.read_data_sets('MNIST_data', one_hot=True)


#定义输入数据mnist图片大小28*28*1=784,None表示batch_size
x = tf.placeholder(dtype=tf.float32,shape=[None,28*28],name="input")
#定义标签数据,mnist共10类
y_ = tf.placeholder(dtype=tf.float32,shape=[None,10],name="y_")
#将数据调整为二维数据，w*H*c---> 28*28*1,-1表示N张
image = tf.reshape(x,shape=[-1,28,28,1])

#第一层，卷积核={5*5*1*32}，池化核={2*2*1,1*2*2*1}
w1 = tf.Variable(initial_value=tf.random_normal(shape=[5,5,1,32],stddev=0.1,dtype=tf.float32,name="w1"))
b1= tf.Variable(initial_value=tf.zeros(shape=[32]))
conv1 = tf.nn.conv2d(input=image,filter=w1,strides=[1,1,1,1],padding="SAME",name="conv1")
relu1 = tf.nn.relu(tf.nn.bias_add(conv1,b1),name="relu1")
pool1 = tf.nn.max_pool(value=relu1,ksize=[1,2,2,1],strides=[1,2,2,1],padding="SAME")
#shape={None，14,14,32}
#第二层，卷积核={5*5*32*64}，池化核={2*2*1,1*2*2*1}
w2 = tf.Variable(initial_value=tf.random_normal(shape=[5,5,32,64],stddev=0.1,dtype=tf.float32,name="w2"))
b2 = tf.Variable(initial_value=tf.zeros(shape=[64]))
conv2 = tf.nn.conv2d(input=pool1,filter=w2,strides=[1,1,1,1],padding="SAME")
relu2 = tf.nn.relu(tf.nn.bias_add(conv2,b2),name="relu2")
pool2 = tf.nn.max_pool(value=relu2,ksize=[1,2,2,1],strides=[1,2,2,1],padding="SAME",name="pool2")
#shape={None，7,7,64}
#FC1
w3 = tf.Variable(initial_value=tf.random_normal(shape=[7*7*64,1024],stddev=0.1,dtype=tf.float32,name="w3"))
b3 = tf.Variable(initial_value=tf.zeros(shape=[1024]))
#关键，进行reshape
input3 = tf.reshape(pool2,shape=[-1,7*7*64],name="input3")
fc1 = tf.nn.relu(tf.nn.bias_add(value=tf.matmul(input3,w3),bias=b3),name="fc1")
#shape={None，1024}
#FC2
w4 = tf.Variable(initial_value=tf.random_normal(shape=[1024,10],stddev=0.1,dtype=tf.float32,name="w4"))
b4 = tf.Variable(initial_value=tf.zeros(shape=[10]))
fc2 = tf.nn.bias_add(value=tf.matmul(fc1,w4),bias=b4)
#shape={None，10}
#定义交叉熵损失
# 使用softmax将NN计算输出值表示为概率
y = tf.nn.softmax(fc2,name="out")

# 定义交叉熵损失函数
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=fc2,labels=y_)
loss = tf.reduce_mean(cross_entropy)
#定义solver
train = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(loss=loss)

#定义正确值,判断二者下标index是否相等
correct_predict = tf.equal(tf.argmax(y,1),tf.argmax(y_,1))
#定义如何计算准确率
accuracy = tf.reduce_mean(tf.cast(correct_predict,dtype=tf.float32),name="accuracy")


#训练NN
with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    for i in range(0,1000):
        xs, ys = mnist.train.next_batch(100)
        session.run(fetches=train,feed_dict={x:xs,y_:ys})
        if i%100 == 0:
            train_accuracy = session.run(fetches=accuracy,feed_dict={x:xs,y_:ys})
            print(i,"accuracy=",train_accuracy)
    #训练完成后，将网络中的权值转化为常量，形成常量graph
    constant_graph = graph_util.convert_variables_to_constants(sess=session,
                                                            input_graph_def=session.graph_def,
                                                            output_node_names=['out'])
    #将带权值的graph序列化，写成pb文件存储起来
    with tf.gfile.FastGFile("lenet.pb", mode='wb') as f:
        f.write(constant_graph.SerializeToString())

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

0 accuracy= 0.05
100 accuracy= 0.88
200 accuracy= 0.92
300 accuracy= 0.96
400 accuracy= 0.99
500 accuracy= 0.9

In [9]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np

mnist = input_data.read_data_sets('MNIST_data',one_hot=True)
pb_path = "lenet.pb"

#导入pb文件到graph中
with tf.gfile.FastGFile(pb_path,'rb') as f:
    # 复制定义好的计算图到新的图中，先创建一个空的图.
    graph_def = tf.GraphDef()
    # 加载proto-buf中的模型
    graph_def.ParseFromString(f.read())
    # 最后复制pre-def图的到默认图中.
    _ = tf.import_graph_def(graph_def, name='')
    
with tf.Session() as session:
    #获取输入tensor
    session.run(tf.global_variables_initializer())
    input_x = tf.get_default_graph().get_tensor_by_name("input:0")
    #获取预测tensor
    output = tf.get_default_graph().get_tensor_by_name("out:0")
    #取第100张图片测试
    one_image = np.reshape(mnist.test.images[100], [-1, 784])
    #将测试图片传入nn中，做inference
    out = session.run(output,feed_dict={input_x:one_image})
    pre_label = np.argmax(out,1)
    print("pre_label=",pre_label)
    print('true label:', np.argmax(mnist.test.labels[100],0))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
pre_label= [6]
true label: 6
